In [ ]:
# default_exp pipeline

# Module pipeline

> Here we have the Nodes wrapping around our core functions, which can be added to a Nipype workflow to process your input images. These definitions here are our custom functions and Nipype Nodes knotted together, which creates a working pipeline.

In [ ]:
#hide
from nbdev.showdoc import *

['test']

In [ ]:
%%capture
#export
import os, sys
from shutil import which

import pipetography.core as ppt

from nipype import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.mrtrix3.utils import BrainMask, MRConvert, DWIExtract, MRMath
from nipype.interfaces.mrtrix3.preprocess import MRDeGibbs
from nipype.interfaces import ants
from nipype.interfaces import fsl
from nipype.interfaces.freesurfer.preprocess import ReconAll

A `pipeline` class that puts everything together.

In [ ]:
# export
class pipeline:
    """
    A class containing functionalities and inputs to the image processing pipeline
    Input: 
        BIDS_dir (str): Valid BIDS directory with DWI modality
    Attributes:
        data_dir (str)
        sub_list (List)
        layout (PyBIDS Layout)
        bfiles_fsl (tuple)
    """

    def __init__(self, BIDS_dir="data"):
        # variables
        self.data_dir = BIDS_dir
        self.sub_list, self.layout = ppt.get_subs(self.data_dir)
        self.dwi_file = os.path.join(
            "sub-{subject_id}", "ses-*", "dwi", "sub-{subject_id}_ses-*_dwi.nii.gz",
        )
        self.b_files = os.path.join(
            "sub-{subject_id}", "ses-1", "dwi", "sub-{subject_id}_ses-1_dwi.bv*"
        )
        self.sub_template = {"dwi": self.dwi_file, "b_files": self.b_files}
        
        # node for reading inputs
        self.sub_source = Node(IdentityInterface(fields=["subject_id"]), name="data_source")
        self.sub_source.iterables = [("subject_id", self.sub_list)]
        
        # node for selecting images and bfiles
        self.select_files = Node(
            SelectFiles(self.sub_template, base_directory=self.data_dir), name="select_files",
        )
        self.bfiles_input = Node(
            Function(
                input_names=["in_List"], output_names=["out_path"], function=ppt.get_bfiles_tuple,
            ),
            name="select_bfiles",
        )
        
        # ACPC HCP MNI152 alignment nodes:
        # first define template location in FSL:
        self.MNI_template = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm.nii.gz')
        self.reduceFOV = MapNode(fsl.utils.RobustFOV(), name="reduce_FOV", iterfield="in_file")
        self.xfminverse = MapNode(fsl.utils.ConvertXFM(), name="transform_inverse", iterfield="in_file")
        self.flirt = MapNode(fsl.preprocess.FLIRT(), name="FLIRT", iterfield="in_file")
        self.concatxfm = MapNode(fsl.utils.ConvertXFM(), name="concat_transform", iterfield=["in_file", "in_file2"])
        self.alignxfm = MapNode(ppt.fslaff2rigid(), name='aff2rigid', iterfield="in_file")
        self.ACPC_warp = MapNode(fsl.preprocess.ApplyWarp(), name='apply_warp', iterfield=["in_file","premat"])
        
        # FreeSurfer recon-all mainly for WM segmentation
        # we are greedy so we just let it segment and produce everything
        self.reconall = MapNode(ReconAll(), name='FSrecon', iterfield="T1_files")
        
        # Preparation nodes (mask, check gradient, convert):
        self.mrconvert = MapNode(ppt.Convert(), name='mrtrix_image', iterfield="in_file")
        self.createMask = MapNode(BrainMask(), name='dwi2mask', iterfield='in_file')
        self.GradCheck = MapNode(ppt.GradCheck(), name='dwigradcheck', iterfield='in_file')
        self.NewGradMR = MapNode(ppt.Convert(), name='mrconvert', iterfield = "in_file")
        
        # preprocessing nodes:
        self.denoise = MapNode(ppt.dwidenoise(), name="denoise", iterfield="in_file")
        self.ringing = MapNode(MRDeGibbs(), name="ringing_removal", iterfield="in_file")
        self.ants_bfc = MapNode(
            ppt.N4BiasFieldCorrection(), name="ants_bias_correct", iterfield="in_file",
        )
        self.mrt_preproc = MapNode(ppt.dwipreproc(), name="mrtrix3_preproc", iterfield="in_file")
        # atlases:
        self.atlas_dir = None
        self.atlas_names = None
        self.atlas_source = None
        self.select_atlas = None
        # b0 volume brain and mask extraction:
        self.b0extract = MapNode(DWIExtract(), name="dwiextract", iterfield="in_file")
        self.b0mean = MapNode(MRMath(), name="mrmath", iterfield="in_file")
        self.fsl_bet = MapNode(fsl.BET(), name="brain_extraction", iterfield="in_file")
        # ants coregistration:
        self.linear_coreg = MapNode(
            ants.Registration(), name="linear_registration", iterfield="fixed_image",
        )
        self.nonlinear_coreg = MapNode(
            ants.Registration(), name="nonlinear_registration", iterfield="fixed_image",
        )
        # data sink:
        self.datasink = Node(DataSink(), name="datasink")
        # workflow for nipype
        self.workflow = None
    
        
    def check_environment(self):
        """
        Check your computing environment for FSL environment variables `FSLOUTPUTTYPE` and `FSLDIR`
        Check if ANTs PATH is included in your environment
        Check if mrtrix3 is in your PATH
        """
        assert "FSLOUTPUTTYPE" in os.environ
        assert "FSLDIR" in os.environ
        if "FSLOUTPUTTYPE" in os.environ:
            print("FSLOUTPUTTYPE is valid")
        else:
            sys.exit("FSLOUTPUTTYPE is not defined, make sure FSL is configured!")

        if "FSLDIR" in os.environ:
            print("FSLDIR is valid")
        else:
            sys.exit("FSLOUTPUTTYPE is not defined, make sure FSL is configured!")
        
        #ANTS:
        assert "ANTSPATH" in os.environ
        print("ANTS is valid")
        #mrtrix:
        assert which('mrview') is not None
        print("mrtrix3 is valid")
        return None
    
        
    def set_datasink(self):
        """
        Set directory where output files will be placed
        """
        output_dir = str(input('Please indicate an output directory: '))
        self.datasink.inputs.base_directory = output_dir
    
    
    def reduceFOV_inputs(
        self,
        xfm_mat="roi2full.mat",
        out_vol="robustfov.nii.gz",
    ):
        """
        Inputs to FSL's robustfov command: robustfov -i [input] -m [matrix name] -r [volume name]
        """
        self.reduceFOV.inputs.out_transform = xfm_mat
        self.reduceFOV.inputs.out_roi = out_vol
    
    
    def flirt_inputs(
        self,
        template=self.MNI_template,
        method='spline',
        mat_file='roi2std.mat',
        out_file='acpc_mni.nii.gz'
    ):
        """
        Inputs to FSL's FLIRT:
        -Iterable input is robustfov.nii.gz (robustFOV's output)
        """
        self.FLIRT.inputs.reference = template
        self.FLIRT.inputs.interp = method
        self.FLIRT.inputs.out_matrix_file = mat_file
        self.FLIRT.inputs.out_file = out_file
    
    
    def xfminverse_inputs(
        self,
        inverse=True,
        out_file='full2roi.mat'
    ):
        """
        Inputs to FSL's convert_xfm command: convert_xfm -omat [filename] -inverse [input]
        """
        self.xfminverse.inputs.out_file = out_file
        self.xfminverse.inputs.invert_xfm = inverse
    
    
    def concatxfm_inputs(
        self,
        concat = True,
        omat = 'full2std.mat'
    ):
        """
        Inputs for transformation matrix concatenation node
        The matrices being concatenated are 1) flirt's output mat file  (in_file2) and 2) inverse mat file from the first convert_xfm (in_file)
        """
        self.concatxfm.inputs.concat_xfm=True
        self.concatxfm.inputs.out_file=omat
        
    
    def aff2rigid_inputs(
        self,
        out_file='outputmatrix'
    ):
        self.alignxfm.inputs.out_file=out_file
    
    
    def acpcwarp_inputs(
        self,
        out_file='acpc_t1.nii.gz',
        rel_warp = True,
        method = 'spline',
        template = self.MNI_template,
    ):
        """
        Inputs for FSL's `applywarp`
        We want spline interpolation, relative warp field, premat = aff2rigid mat output.
        """
        self.ACPC_warp.inputs.out_file = out_file
        self.ACPC_warp.inputs.relwarp = rel_warp
        self.ACPC_warp.inputs.interp = method
        self.ACPC_warp.inputs.ref_file = template
        
    
    # Freesurfer recon-all
    def recon_inputs(
        self,
        hipp = True,
    ):
        """
        We will multiproc the entire workflow, so parallel here is not recommended.
        Only recommend parallel for single subject runs or group level runs.
        """
        self.reconall.inputs.directive='all' # we always want recon-all -all
        self.reconall.inputs.subjects_dir = self.BIDS_dir+'/derivatives/freesurfer'
        self.reconall.inputs.hippocampal_subfields_T1 = True # we want this for compeleteness of data
        #self.reconall.inputs.subject_id
        
    
    # Convert to mrtrix3 file format:
    def mrconvert_inputs(
        self,
        out_file = 'raw_dwi.mif',
        export_mrtrix3_grad = True,
        out_bfile = 'raw_dwi.b',
        force = True
    ):
        """
        Convert a DWI to mrtrix3's image format and gradient files
        Workflow connected inputs not listed here:
            - grad_fsl - gradient files in fsl format
            - in_file - raw dwi that you want to convert to mrtrix3 format
        """
        self.mrconvert.inputs.out_file = out_file
        self.mrconvert.inputs.export_grad = export_mrtrix3_grad
        self.mrconvert.inputs.out_bfile = out_bfile
        self.mrconvert.inputs.force = force
    
    
    # DWI Preprocessing mask creation:
    def createMask_inputs(
        self,
        out_file = 'b0_brain_mask.mif'
    ):
        """
        Create brain mask from B0 DWI volumes for preprocessing tasks.
        in_file will be connected in workflow
        """
        self.createMask.inputs.out_file = out_file
        
    
    # gradient check and create files with corrected gradient:
    def GradCheck_inputs(
        self,
        out_bfile = 'corrected.b',
        export_mrtrix3_grad = True,
        force = True
    ):
        """
        Set inputs to dwigradcheck
        Workflow inputs:
            in_file = input from mrconvert Node 
            mask_file = input from createMask Node
            grad_file = input from mrconvert Node
        """
        self.GradCheck.inputs.export_grad = export_mrtrix3_grad
        self.GradCheck.inputs.out_bfile = out_bfile
        self.GradCheck.inputs.force = force
    
    
    # Create new dwi image with corrected gradients:
    def NewGradMR_inputs(
        self,
        out_file = 'cdwi.mif'
        force = True
    ):
        """
        Inputs for creating gradient corrected MR
        Outputs a correced dwi file
        Workflow connected inputs:
            in_file - output from mrconvert Node
            grad_file - output from GradCheck Node (out_bfile)
        """
        self.NewGradMR.inputs.out_file = out_file
        self.NewGradMR.inputs.force = force
        
    
    # MPPCA Denoise with dwidenoise from mrtrix3
    def denoise_inputs(
        self,
        out_file="denoised.nii.gz",
        noise_file="noise_map.nii.gz",
        force=True,
        quiet=False,
        nthreads=None,
    ):
        """
        Set inputs to denoise node
        """
        self.denoise.inputs.out_file = out_file
        self.denoise.inputs.noise = noise_file
        self.denoise.inputs.force = force

        if quiet is True:
            self.denoise.inputs.quiet = "-quiet"
        else:
            pass

        if nthreads is None:
            pass
        elif type(nthreads) == int:
            self.denoise.inputs.nthreads = nthreads
        else:
            sys.exit("denoise nthreads input has to be an integer")

            
    # Gibbs ringing artifact removal
    def DeGibbs_inputs(self, out_file="ringing_removed.nii.gz"):
        """
        Set inputs to ringing removal node
        """
        self.ringing.inputs.out_file = out_file

        
        
    def ants_bfc_inputs(self, out_file="biasfieldcorrected.nii.gz", dims=4):
        """
        Set inputs to ANTs bias field correction node, currently incomplete
        """
        self.ants_bfc.inputs.out_file = out_file
        self.ants_bfc.inputs.dims = dims

    def mrt_preproc_inputs(
        self, rpe_options, pe_dir, eddy_options, nthreads, out_file="preproc.nii.gz",
    ):
        """
        Set inputs to mrtrix3's dwipreproc.
        Arguments:
            out_file: name of output file
            rpe_options (str): specify phase encoding design -rpe_+ 'none/pair/all/header'
            pe_dir (str): phase encoding direction(i,j,k)/(0,1,2)/(RL,PA,IS)
            eddy_options (str): '"--slm=linear "' aditional commandline options to eddy command, with at least 1  space
        """
        self.mrt_preproc.inputs.out_file = out_file
        self.mrt_preproc.inputs.rpe_options = rpe_options  # -rpe_none
        self.mrt_preproc.inputs.pe_dir = pe_dir
        self.mrt_preproc.inputs.eddy_options = eddy_options
        if nthreads is None:
            pass
        elif type(nthreads) == int:
            self.mrt_preproc.inputs.nthreads = nthreads
        else:
            sys.exit("denoise nthreads input has to be an integer")

    def atlas_inputs(self, atlas_dir, atlas_names):
        """
        Input where to load atlases for ROIs and co-registration
        """
        self.atlas_dir = atlas_dir
        self.atlas_names = atlas_names
        atlas_template = {"atlas": atlas_dir + "{file_name}"}
        self.atlas_source = Node(IdentityInterface(fields=["atlas_name"]), name="atlas_source")
        self.atlas_source.iterables = [("atlas_name"), self.atlas_names]
        self.select_atlas = Node(SelectFiles(atlas_template), name="select_atlases")
        self.select_atlas.base_directory = self.atlas_dir

    def b0extract_inputs(self, out_file="dwi_b0.nii.gz", bzero=True):
        self.b0extract.inputs.out_file = out_file
        self.b0extract.inputs.bzero = bzero

    def b0mean_inputs(self, out_file="dwi_b0_mean.nii.gz", axis=3):
        self.b0mean.inputs.operation = "mean"
        self.b0mean.inputs.axis = axis
        self.b0mean.inputs.out_file = out_file

    def bet_inputs(
        self, robust=True, mask=True, frac=0.25, out_file="dwi_brain.nii.gz"
    ):
        if robust is True:
            self.fsl_bet.inputs.robust = robust
        else:
            self.fsl_bet.inputs.reduce_bias = False

        self.fsl_bet.inputs.mask = mask
        self.fsl_bet.inputs.frac = 0.25
        self.fsl_bet.inputs.out_file = out_file

    def linear_coreg_inputs(
        self,
        dims=3,
        prefix="atlas_in_dwi_affine",
        collapse=True,
        transform=["Affine"],
        parameters=[(0.1,)],
        metric=["MI"],
        weight=[1],
        radius_bins=[64],
        num_iter=[[500, 200, 200, 100]],
        threshold=[1e-06],
        window_size=[10],
        sigmas=[[4, 2, 1, 0]],
        units=["vox"],
        shrink=[[8, 4, 2, 1]],
        histogram=True,
        output="atlas_in_dwi_affine.nii.gz",
    ):
        self.linear_coreg.inputs.dimension = dims
        self.linear_coreg.inputs.output_transform_prefix = prefix
        self.linear_coreg.inputs.collapse_output_transforms = collapse
        self.linear_coreg.inputs.transforms = transform
        self.linear_coreg.inputs.transform_parameters = parameters
        self.linear_coreg.inputs.metric = metric
        self.linear_coreg.inputs.metric_weight = weight  # default, value ignored by ANTS
        self.linear_coreg.inputs.radius_or_number_of_bins = radius_bins
        # -convergence
        self.linear_coreg.inputs.number_of_iterations = num_iter
        self.linear_coreg.inputs.convergence_threshold = threshold
        self.linear_coreg.inputs.convergence_window_size = window_size
        # -s
        self.linear_coreg.inputs.smoothing_sigmas = sigmas
        self.linear_coreg.inputs.sigma_units = units
        # -f
        self.linear_coreg.inputs.shrink_factors = shrink
        self.linear_coreg.inputs.use_histogram_matching = histogram  # -u flag
        self.linear_coreg.inputs.output_warped_image = output
    
    def nonlinear_coreg_inputs(
        self,
        dims=3,
        prefix="atlas_in_dwi_syn",
        collapse=True,
        transform=['SyN'],
        parameters=[(0.1,)],
        metric=["MI"],
        weight=[1],
        radius_bins=[64],
        num_iter=[[500, 200, 200, 100]],
        threshold=[1e-06],
        window_size=[10],
        sigmas=[[4, 2, 1, 0]],
        units=["vox"],
        shrink=[[8, 4, 2, 1]],
        histogram=True,
        output="atlas_in_dwi_syn.nii.gz",
    ):
        self.nonlinear_coreg.inputs.dimension = dims
        self.nonlinear_coreg.inputs.output_transform_prefix = prefix
        self.nonlinear_coreg.inputs.collapse_output_transforms = collapse
        self.nonlinear_coreg.inputs.transforms = transform
        self.nonlinear_coreg.inputs.transform_parameters = parameters
        self.nonlinear_coreg.inputs.metric = metric
        self.nonlinear_coreg.inputs.metric_weight = weight  # default, value ignored by ANTS
        self.nonlinear_coreg.inputs.radius_or_number_of_bins = radius_bins
        # -convergence
        self.nonlinear_coreg.inputs.number_of_iterations = num_iter
        self.nonlinear_coreg.inputs.convergence_threshold = threshold
        self.nonlinear_coreg.inputs.convergence_window_size = window_size
        # -s
        self.nonlinear_coreg.inputs.smoothing_sigmas = sigmas
        self.nonlinear_coreg.inputs.sigma_units = units
        # -f
        self.nonlinear_coreg.inputs.shrink_factors = shrink
        self.nonlinear_coreg.inputs.use_histogram_matching = histogram  # -u flag
        self.nonlinear_coreg.inputs.output_warped_image = output

    def connect_nodes(self, wf_name="pipetography"):
        # a workflow that will be filled in:
        self.workflow = Workflow(name=wf_name, base_dir=self.data_dir + "/derivatives",)
        # folder -> select files
        self.workflow.connect(
            self.sub_source, "subject_id", self.select_files, "subject_id",
        )
        # select files -> obtain bvec/bval path as tuple
        self.workflow.connect(
            self.select_files, "b_files", self.bfiles_input, "in_List",
        )
        # selected dwi -> denoise
        self.workflow.connect(
            self.select_files, "dwi", self.denoise, "in_file",
        )
        # denoised output -> datasink
        self.workflow.connect(
            self.denoise, "out_file", self.datasink, "preproc.@denoised",
        )
        # denoised output -> ringing removal
        self.workflow.connect(
            self.denoise, "out_file", self.ringing, "in_file",
        )
        # ringing removal -> datasink
        self.workflow.connect(
            self.ringing, "out_file", self.datasink, "preproc.@ringing_removed",
        )
        # ringing removal -> bias field correction
        self.workflow.connect(
            self.ringing, "out_file", self.ants_bfc, "in_file",
        )
        # bias field corrected -> datasink
        self.workflow.connect(
            self.ants_bfc, "out_file", self.datasink, "preproc.@biasCorrected",
        )
        # bias field corrected -> eddy correction
        self.workflow.connect(
            self.ants_bfc, "out_file", self.mrt_preproc, "in_file",
        )
        # grad_fsl for eddy correction:
        self.workflow.connect(
            self.bfiles_input, "out_path", self.mrt_preproc, "grad_fsl",
        )
        # preproc --> data sink
        self.workflow.connect(
            self.mrt_preproc, "out_file", self.datasink, "preproc.@preproced",
        )
        # b0 extractions:
        self.workflow.connect(
            self.mrt_preproc, "out_file", self.b0extract, "in_file",
        )
        self.workflow.connect(
            self.bfiles_input, "out_path", self.b0extract, "grad_fsl",
        )
        self.workflow.connect(
            self.b0extract, "out_file", self.b0mean, "in_file",
        )
        self.workflow.connect(
            self.b0mean, "out_file", self.fsl_bet, "in_file",
        )
        # extractions --> datasink:
        self.workflow.connect(
            self.b0extract, "out_file", self.datasink, "extraction.@volume",
        )
        self.workflow.connect(
            self.b0mean, "out_file", self.datasink, "extraction.@mean",
        )
        self.workflow.connect(
            self.fsl_bet, "mask_file", self.datasink, "extraction.@BET_mask",
        )
        self.workflow.connect(
            self.fsl_bet, "out_file", self.datasink, "extraction.@BET_image",
        )
        # coregistration of atlases to DWI, select atlases as moving image
        self.workflow.connect(
            self.atlas_source, 'atlas_name', self.select_atlas, 'file_name'
        )
        self.workflow.connect(
            self.select_atlas, 'atlas', self.linear_coreg, 'moving_image'
        )
        # use extracted brain as target brain for co-registration, no dwi noise
        self.workflow.connect(
            self.fsl_bet, 'out_file', self.linear_coreg, 'fixed_image'
        )
        self.workflow.connect(
            self.fsl_bet, 'out_file', self.nonlinear_coreg, 'fixed_image'
        )
        self.workflow.connect(
            self.linear_coreg, 'warped_image', self.nonlinear_coreg, 'moving_image'
        )
        # coreg --> datasink
        self.workflow.connect(
            self.linear_coreg, 'warped_image', self.datasink, 'coreg.@affine_atlas'
        )
        self.workflow.connect(
            self.nonlinear_coreg, 'warped_image', self.datasink, 'coreg.@syn_atlas'
        )
        #
        
    def default_setup(self, phase_encoding_design = "-rpe_none", phase_encoding_dir = "j-"):
        """
        Set up pipeline parameters with all default settings blindly
        You want to use this for a quick test run. Then tweak parameters after examining all output images
        Note that this assumes input options for eddy current correction as "--slm=linear" 
        Arguments:
            phase_encoding_design (str): defaults to "-rpe_none", or no reverse phase encoding volume, eddy current and motion correction only, other options include "rpe_pair, rpe_all, rpe_header"
            phase_encoding_dir (str): defaults to "j-", or anterior to posterior. (Options: i,j,k or RL, PA, IS)
        """
        self.denoise_inputs(force = True, quiet = True)
        self.DeGibbs_inputs()
        self.ants_bfc_inputs()
        self.mrt_preproc_inputs(
            rpe_options=phase_encoding_design,
            pe_dir=phase_encoding_dir,
            eddy_options='"--slm=linear --verbose"',
            nthreads=4,
            )
        self.b0extract_inputs()
        self.b0mean_inputs()
        self.bet_inputs()
        self.atlas_inputs(atlas_dir=str(input('Please indicate directory with atlas volumes: ')), atlas_names=list(input('Please indicate list of selected atlas names: ')))
        self.linear_coreg_inputs() #defaults
        self.nonlinear_coreg_inputs() #defaults
        self.connect_nodes(wf_name = 'default_workflow')
        return print(self.__dict__)
        
    def draw_pipeline(self):
        self.workflow.write_graph(graph2use='orig', dotfilename = 'pipetography.dot')
        
    def run_pipeline(self, parallel = True):
        if parallel is True:
            processes = int(input('Number of Processes: '))
            self.workflow.run('MultiProc', plugin_args = {'n_procs': processes})

Example:

In [ ]:
#create pipeline:
preproc_dwi = pipeline()

Creating layout of data directory, might take a while if there are a lot of subjects


Check if our environment is properly set-up:

In [ ]:
#example
preproc_dwi.check_environment()

FSLOUTPUTTYPE is valid
FSLDIR is valid
ANTS is valid
mrtrix3 is valid


In [ ]:
#example
#set output destination:
preproc_dwi.set_datasink()

Please indicate an output directory:  'output'


Take a look at what's in the `pipeline`:

In [ ]:
preproc_dwi.__dict__

{'data_dir': 'data',
 'sub_list': ['11048'],
 'layout': BIDS Layout: ...ers/xxie/lab/pipetography/data | Subjects: 1 | Sessions: 1 | Runs: 0,
 'dwi_file': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz',
 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*',
 'sub_template': {'dwi': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz',
  'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*'},
 'sub_source': data_source,
 'select_files': select_files,
 'bfiles_input': select_bfiles,
 'denoise': denoise,
 'ringing': ringing_removal,
 'ants_bfc': ants_bias_correct,
 'mrt_preproc': mrtrix3_preproc,
 'atlas_dir': None,
 'atlas_names': None,
 'atlas_source': None,
 'select_atlas': None,
 'b0extract': dwiextract,
 'b0mean': mrmath,
 'fsl_bet': brain_extraction,
 'linear_coreg': linear_registration,
 'nonlinear_coreg': nonlinear_registration,
 'datasink': datasink,
 'workflow': None}

We can set up preprocessing pipeline with default parameters:

In [ ]:
#example
#usage
preproc_dwi.default_setup()

Please indicate directory with atlas volumes:  '/Users/xxie/lab/atlases'
Please indicate list of selected atlas names:  ['BN_Atlas_246_2mm.nii','DK_atlas86_1mm.nii']


{'data_dir': 'data', 'sub_list': ['11048'], 'layout': BIDS Layout: ...ers/xxie/lab/pipetography/data | Subjects: 1 | Sessions: 1 | Runs: 0, 'dwi_file': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz', 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*', 'sub_template': {'dwi': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz', 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*'}, 'sub_source': default_workflow.data_source, 'select_files': default_workflow.select_files, 'bfiles_input': default_workflow.select_bfiles, 'denoise': default_workflow.denoise, 'ringing': default_workflow.ringing_removal, 'ants_bfc': default_workflow.ants_bias_correct, 'mrt_preproc': default_workflow.mrtrix3_preproc, 'atlas_dir': "'/Users/xxie/lab/atlases'", 'atlas_names': ['[', "'", 'B', 'N', '_', 'A', 't', 'l', 'a', 's', '_', '2', '4', '6', '_', '2', 'm', 'm', '.', 'n', 'i', 'i', "'", ',', "'", 'D', 'K', '_', 'a', 't', 'l', 'a', 's', '8', '6', 

OR we can fill this in one by one:

First, let's tell the pipeline where we have atlas volumes and which ones to use:

In [ ]:
#example
preproc_dwi.atlas_inputs(atlas_dir = '/Users/xxie/lab/atlases', atlas_names = ['BN_Atlas_246_2mm.nii','DK_atlas86_1mm.nii'])

Next, we give inputs to the `denoise` Node:

In [ ]:
#example
preproc_dwi.denoise_inputs(force = True, quiet = True)
# we kept everything else default, like output names and number of threads.

Gibbs ringing removal:

In [ ]:
#example
preproc_dwi.DeGibbs_inputs() # keep it default
print('Output file name is ' + preproc_dwi.ringing.inputs.out_file)

ANTs Bias Field Correction:

In [ ]:
#example
preproc_dwi.ants_bfc_inputs() # keep it default:
preproc_dwi.ants_bfc.inputs.print_traits

`in_file` is undefined because we will feed it this information once we connect our individual functions.

Next let's set up eddy current/motion correction, we need to tell the pipeline the phase encoding settings and inputs to eddy algorithm:

In [ ]:
#example
preproc_dwi.mrt_preproc_inputs(
    rpe_options="-rpe_none",
    pe_dir="j-",
    eddy_options='"--slm=linear --verbose"',
    nthreads=4,
)
preproc_dwi.mrt_preproc.inputs.print_traits

`grad_fsl` and `in_file` will be input via connected workflow later

Next, we set-up brain extraction from B0 volumes:

In [ ]:
#example
# Extract b0 volumes:
preproc_dwi.b0extract_inputs() #default
# Create average B0 volume:
preproc_dwi.b0mean_inputs() #default
# Extract brain from B0 average volume:
preproc_dwi.bet_inputs() #defaults again, using FSL's BET

ANTs registration set up:

In [ ]:
#example
preproc_dwi.linear_coreg_inputs() #defaults
preproc_dwi.nonlinear_coreg_inputs() #defaults
preproc_dwi.nonlinear_coreg.inputs.print_traits

All the default settings may not be optimal for your dataset, run the processing for a test image and see the intermediate results and then tweak the available inputs to improve the output image quality.

Now that our pre-processing Nodes are properly setup, we can connect and create a workflow:

In [ ]:
#example
preproc_dwi.connect_nodes(wf_name = 'pipetography_workflow')
preproc_dwi.draw_pipeline()

Let's take a look at the final workflow we created:

In [ ]:
#example
from IPython.display import Image
Image('data/derivatives/test_run/pipetography_detailed.png')